In [0]:
!pip install torch

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
!pip install torchvision

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
%fs ls FileStore/tables/pneumonia/train/PNEUMONIA

path,name,size,modificationTime
dbfs:/FileStore/tables/pneumonia/train/PNEUMONIA/person1000_bacteria_2931.jpeg,person1000_bacteria_2931.jpeg,69942,1701118577000
dbfs:/FileStore/tables/pneumonia/train/PNEUMONIA/person1000_virus_1681.jpeg,person1000_virus_1681.jpeg,68494,1701118577000
dbfs:/FileStore/tables/pneumonia/train/PNEUMONIA/person1001_bacteria_2932.jpeg,person1001_bacteria_2932.jpeg,105426,1701118577000
dbfs:/FileStore/tables/pneumonia/train/PNEUMONIA/person1002_bacteria_2933.jpeg,person1002_bacteria_2933.jpeg,106265,1701118577000
dbfs:/FileStore/tables/pneumonia/train/PNEUMONIA/person1003_bacteria_2934.jpeg,person1003_bacteria_2934.jpeg,116695,1701118578000
dbfs:/FileStore/tables/pneumonia/train/PNEUMONIA/person1003_virus_1685.jpeg,person1003_virus_1685.jpeg,53328,1701118578000
dbfs:/FileStore/tables/pneumonia/train/PNEUMONIA/person1004_bacteria_2935.jpeg,person1004_bacteria_2935.jpeg,53005,1701118578000
dbfs:/FileStore/tables/pneumonia/train/PNEUMONIA/person1004_virus_1686.jpeg,person1004_virus_1686.jpeg,39690,1701118578000
dbfs:/FileStore/tables/pneumonia/train/PNEUMONIA/person1005_bacteria_2936.jpeg,person1005_bacteria_2936.jpeg,56219,1701118579000
dbfs:/FileStore/tables/pneumonia/train/PNEUMONIA/person1005_virus_1688.jpeg,person1005_virus_1688.jpeg,74882,1701118579000


In [0]:
import os
import torch
from torchvision import models, datasets, transforms
import torch.nn as nn
import numpy as np
import torch.optim as optim
from sklearn.metrics import accuracy_score, average_precision_score, balanced_accuracy_score
import mlflow
import mlflow.pytorch
from torchvision.models import mobilenet_v2, MobileNet_V2_Weights
from mlflow.models import infer_signature
from sklearn.model_selection import train_test_split

In [0]:
def initialize_model(weight, freeze_except_last_n_layers, add_loss_criteria_weights):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    if weight == 'IMAGENET1K_V1':
        model = models.mobilenet_v2(weights=MobileNet_V2_Weights.IMAGENET1K_V1)
    elif weight == 'IMAGENET1K_V2':
        model = models.mobilenet_v2(weights=MobileNet_V2_Weights.IMAGENET1K_V2) 
    else:
        # Handle unexpected 'weight' value
        raise ValueError(f"Unsupported weight value: {weight}")       

    # Freeze all layers except the last n layers
    if freeze_except_last_n_layers > 0:
        layers = list(model.children())
        for layer in layers[:-freeze_except_last_n_layers]:
            for param in layer.parameters():
                param.requires_grad = False

    num_ftrs = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(num_ftrs, 2)

    if add_loss_criteria_weights == True:
        loss_weights = torch.Tensor([2, 1]).to(device)
        criterion = nn.CrossEntropyLoss(weight=loss_weights)
    else:
        criterion = nn.CrossEntropyLoss()
    
    return model, criterion

In [0]:
def prepare_data(batch_size, data_transforms, data_dir, val_size=0.2):
    # Load datasets
    train_data = datasets.ImageFolder(os.path.join(data_dir, 'train'), data_transforms['train'])
    val_data = datasets.ImageFolder(os.path.join(data_dir, 'val'), data_transforms['val'])
    test_data = datasets.ImageFolder(os.path.join(data_dir, 'test'), data_transforms['test'])

    # Combine validation and test data
    combined_val_test_data = torch.utils.data.ConcatDataset([val_data, test_data])

    # Split the original training data into new training and validation sets
    train_indices, val_indices = train_test_split(list(range(len(train_data))), test_size=val_size)
    new_train_data = torch.utils.data.Subset(train_data, train_indices)
    new_val_data = torch.utils.data.Subset(train_data, val_indices)

    # Create data loaders
    dataloaders = {
        'train': torch.utils.data.DataLoader(new_train_data, batch_size=batch_size, shuffle=True),
        'val': torch.utils.data.DataLoader(new_val_data, batch_size=batch_size, shuffle=False),
        'test': torch.utils.data.DataLoader(combined_val_test_data, batch_size=batch_size, shuffle=False)
    }

    return dataloaders

In [0]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=10, device=None, patience=3):
    all_pred_labels = []
    all_ground_labels = []
    best_val_loss = float('inf')
    epochs_no_improve = 0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        print('---------------')

        # Training Phase
        model.train()
        train_loss = 0
        total_train_batches = 0

        for train_data, train_labels in dataloaders['train']:
            optimizer.zero_grad()
            out = model(train_data.to(device))
            loss = criterion(out, train_labels.to(device))
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            total_train_batches += 1

        print('Train loss: {:.2f}'.format(train_loss / total_train_batches))

        # Validation Phase
        model.eval()
        val_loss = 0
        total_val_batches = 0

        with torch.no_grad():
            for val_data, val_labels in dataloaders['val']:
                out = model(val_data.to(device))
                loss = criterion(out, val_labels.to(device))
                val_loss += loss.item()
                total_val_batches += 1

        current_val_loss = val_loss / total_val_batches
        print('Val loss: {:.2f}'.format(current_val_loss))
        
        actual_epochs = epoch + 1  # Update the number of actual epochs

        if current_val_loss < best_val_loss:
            best_val_loss = current_val_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= patience:
            print('Early stopping triggered after {} epochs without improvement'.format(patience))
            break

    # Testing Phase after all epochs
    model.eval()
    test_loss = 0
    total_test_batches = 0

    with torch.no_grad():
        for test_data, test_labels in dataloaders['test']:
            out = model(test_data.to(device))
            loss = criterion(out, test_labels.to(device))
            test_loss += loss.item()
            output = torch.argmax(torch.nn.functional.softmax(out, dim=1), dim=1).cpu().detach().numpy()

            for i in range(output.shape[0]):
                all_pred_labels.append(output[i])
                all_ground_labels.append(test_labels.cpu().detach().numpy()[i])

    accuracy = accuracy_score(np.array(all_ground_labels), np.array(all_pred_labels))
    avg_precision = average_precision_score(np.array(all_ground_labels), np.array(all_pred_labels))
    balanced_acc = balanced_accuracy_score(np.array(all_ground_labels), np.array(all_pred_labels))

    return accuracy, avg_precision, balanced_acc, actual_epochs

In [0]:
def main(batch_size, num_epochs, learning_rate, weight, freeze_except_last_n_layers, add_loss_criteria_weights, exp_name):
    with mlflow.start_run(run_name= exp_name):
        # Data transformations
        data_transforms = {
            'train': transforms.Compose([
                transforms.Resize(255),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ]),
            'val': transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ]),
            'test': transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])
        }

        # Log hyperparameters
        mlflow.log_param("batch_size", batch_size)
        mlflow.log_param("num_epochs", num_epochs)
        mlflow.log_param("freeze_except_last_n_layers", freeze_except_last_n_layers)
        mlflow.log_param("add_loss_criteria_weights", add_loss_criteria_weights)
        mlflow.log_param("weight", weight)

        # Initialize model, dataloaders, optimizer, etc.
        model, criterion = initialize_model(weight, freeze_except_last_n_layers, add_loss_criteria_weights)
        
        destination_path = '/dbfs/FileStore/tables/pneumonia/'
        dataloaders = prepare_data(batch_size, data_transforms, destination_path, val_size=0.2)
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        model.to(device)

        # Generate Sample Data and Convert to NumPy
        sample_data, _ = next(iter(dataloaders['train']))
        sample_data = sample_data.to(device)

        # Predict on Sample Data to get Sample Output and Convert to NumPy
        model.eval()  # Set the model to evaluation mode
        with torch.no_grad():
            sample_output = model(sample_data)
            sample_input_np = sample_data.cpu().numpy()
            sample_output_np = sample_output.cpu().numpy()

        # Infer Signature with NumPy Arrays
        signature = infer_signature(sample_input_np, sample_output_np)

        optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate)

        # Train model and get metrics
        accuracy, avg_precision, balanced_acc, actual_epochs_completed = train_model(model, dataloaders, criterion, optimizer, num_epochs, device)

        mlflow.log_param("actual_epochs_completed", actual_epochs_completed)

        # Log metrics
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("average_precision", avg_precision)
        mlflow.log_metric("balanced_accuracy", balanced_acc)

        # Log the PyTorch model
        mlflow.pytorch.log_model(model, "model", signature=signature)

In [0]:
# Experiment 4_1
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 50, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 0, add_loss_criteria_weights = False, exp_name = 'MobileNetV2 E_4_1')

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File <command-1652671916370585>, line 3
      1 # Experiment 4_1
      2 if __name__ == '__main__':
----> 3     main(weight = 'IMAGENET1K_V1', batch_size = 50, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 0, add_loss_criteria_weights = False, exp_name = 'MobileNetV2 E_8_1')

File <command-1652671916370584>, line 2, in main(batch_size, num_epochs, learning_rate, weight, freeze_except_last_n_layers, add_loss_criteria_weights, exp_name)
      1 def main(batch_size, num_epochs, learning_rate, weight, freeze_except_last_n_layers, add_loss_criteria_weights, exp_name):
----> 2     with mlflow.start_run(run_name= exp_name):
      3         # Data transformations
      4         data_transforms = {
      5             'train': transforms.Compose([
      6                 transforms.Resize(255),
   (...)
     20          

In [0]:
# Experiment 4_2
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 100, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 0, add_loss_criteria_weights = False, exp_name = 'MobileNetV2 E_4_2')

MobileNetV2 E_4_2


---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File <command-2251866119748019>, line 3
      1 # Experiment 4_2
      2 if __name__ == '__main__':
----> 3     main(weight = 'IMAGENET1K_V1', batch_size = 100, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 0, add_loss_criteria_weights = False, exp_name = 'MobileNetV2 E_4_2')

File <command-1652671916370584>, line 3, in main(batch_size, num_epochs, learning_rate, weight, freeze_except_last_n_layers, add_loss_criteria_weights, exp_name)
      1 def main(batch_size, num_epochs, learning_rate, weight, freeze_except_last_n_layers, add_loss_criteria_weights, exp_name):
      2     print(exp_name)
----> 3     with mlflow.start_run(run_name= exp_name):
      4         # Data transformations
      5         data_transforms = {
      6             'train': transforms.Compose([
      7                 transforms.Resize(255

In [0]:
# Experiment 4_3
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 150, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 0, add_loss_criteria_weights = False, exp_name = 'MobileNetV2 E_4_3')

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File <command-708418984708533>, line 3
      1 # Experiment 4_3
      2 if __name__ == '__main__':
----> 3     main(weight = 'IMAGENET1K_V1', batch_size = 150, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 0, add_loss_criteria_weights = False, exp_name = 'MobileNetV2 E_4_3')

File <command-1652671916370584>, line 2, in main(batch_size, num_epochs, learning_rate, weight, freeze_except_last_n_layers, add_loss_criteria_weights, exp_name)
      1 def main(batch_size, num_epochs, learning_rate, weight, freeze_except_last_n_layers, add_loss_criteria_weights, exp_name):
----> 2     with mlflow.start_run(run_name= exp_name):
      3         # Data transformations
      4         data_transforms = {
      5             'train': transforms.Compose([
      6                 transforms.Resize(255),
   (...)
     20          

In [0]:
# Experiment 4_4
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 200, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 0, add_loss_criteria_weights = False, exp_name = 'MobileNetV2 E_4_4')

Epoch 1/15
---------------
Train loss: 0.13
Val loss: 2.11
Epoch 2/15
---------------
Train loss: 0.06
Val loss: 0.47
Epoch 3/15
---------------
Train loss: 0.02
Val loss: 0.63
Epoch 4/15
---------------
Train loss: 0.01
Val loss: 0.66
Epoch 5/15
---------------
Train loss: 0.01
Val loss: 2.54
Epoch 6/15
---------------
Train loss: 0.02
Val loss: 0.63
Epoch 7/15
---------------
Train loss: 0.04
Val loss: 0.81
Epoch 8/15
---------------
Train loss: 0.01
Val loss: 1.49
Epoch 9/15
---------------
Train loss: 0.01
Val loss: 0.69
Epoch 10/15
---------------
Train loss: 0.00
Val loss: 0.91
Epoch 11/15
---------------
Train loss: 0.02
Val loss: 1.33
Epoch 12/15
---------------
Train loss: 0.01
Val loss: 0.96
Epoch 13/15
---------------
Train loss: 0.00
Val loss: 1.68
Epoch 14/15
---------------
Train loss: 0.00
Val loss: 1.32
Epoch 15/15
---------------
Train loss: 0.00
Val loss: 1.21


2023/11/28 22:12:56 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.7.1/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.


Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]

In [0]:
# Experiment 4_5
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 50, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 2, add_loss_criteria_weights = False, exp_name = 'MobileNetV2 E_4_5')

Epoch 1/15
---------------
Train loss: 0.12
Val loss: 0.36
Epoch 2/15
---------------
Train loss: 0.05
Val loss: 0.93
Epoch 3/15
---------------
Train loss: 0.05
Val loss: 1.47
Epoch 4/15
---------------
Train loss: 0.04
Val loss: 1.51
Epoch 5/15
---------------
Train loss: 0.02
Val loss: 0.71
Epoch 6/15
---------------
Train loss: 0.03
Val loss: 0.73
Epoch 7/15
---------------
Train loss: 0.02
Val loss: 1.77
Epoch 8/15
---------------
Train loss: 0.02
Val loss: 1.49
Epoch 9/15
---------------
Train loss: 0.02
Val loss: 3.96
Epoch 10/15
---------------
Train loss: 0.02
Val loss: 1.02
Epoch 11/15
---------------
Train loss: 0.02
Val loss: 0.68
Epoch 12/15
---------------
Train loss: 0.02
Val loss: 0.92
Epoch 13/15
---------------
Train loss: 0.01
Val loss: 0.83
Epoch 14/15
---------------
Train loss: 0.00
Val loss: 1.11
Epoch 15/15
---------------
Train loss: 0.01
Val loss: 1.35


2023/11/28 22:55:46 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.7.1/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.


Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]

In [0]:
# Experiment 4_6
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 100, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 2, add_loss_criteria_weights = False, exp_name = 'MobileNetV2 E_4_6')

Epoch 1/15
---------------
Train loss: 0.11
Val loss: 0.28
Epoch 2/15
---------------
Train loss: 0.04
Val loss: 1.49
Epoch 3/15
---------------
Train loss: 0.06
Val loss: 0.20
Epoch 4/15
---------------
Train loss: 0.03
Val loss: 0.59
Epoch 5/15
---------------
Train loss: 0.02
Val loss: 2.16
Epoch 6/15
---------------
Train loss: 0.05
Val loss: 1.36
Epoch 7/15
---------------
Train loss: 0.03
Val loss: 0.46
Epoch 8/15
---------------
Train loss: 0.01
Val loss: 0.92
Epoch 9/15
---------------
Train loss: 0.01
Val loss: 0.54
Epoch 10/15
---------------
Train loss: 0.00
Val loss: 1.39
Epoch 11/15
---------------
Train loss: 0.00
Val loss: 1.27
Epoch 12/15
---------------
Train loss: 0.00
Val loss: 1.31
Epoch 13/15
---------------
Train loss: 0.01
Val loss: 0.43
Epoch 14/15
---------------
Train loss: 0.04
Val loss: 2.73
Epoch 15/15
---------------
Train loss: 0.08
Val loss: 0.46


2023/11/28 23:38:45 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.7.1/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.


Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]

In [0]:
# Experiment 4_7
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 150, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 2, add_loss_criteria_weights = False, exp_name = 'MobileNetV2 E_4_7')

Epoch 1/15
---------------
Train loss: 0.12
Val loss: 1.25
Epoch 2/15
---------------
Train loss: 0.04
Val loss: 0.49
Epoch 3/15
---------------
Train loss: 0.03
Val loss: 0.58
Epoch 4/15
---------------
Train loss: 0.01
Val loss: 0.43
Epoch 5/15
---------------
Train loss: 0.02
Val loss: 0.63
Epoch 6/15
---------------
Train loss: 0.01
Val loss: 0.76
Epoch 7/15
---------------
Train loss: 0.00
Val loss: 1.26
Epoch 8/15
---------------
Train loss: 0.00
Val loss: 2.33
Epoch 9/15
---------------
Train loss: 0.01
Val loss: 1.10
Epoch 10/15
---------------
Train loss: 0.01
Val loss: 1.62
Epoch 11/15
---------------
Train loss: 0.02
Val loss: 0.81
Epoch 12/15
---------------
Train loss: 0.00
Val loss: 1.71
Epoch 13/15
---------------
Train loss: 0.00
Val loss: 0.59
Epoch 14/15
---------------
Train loss: 0.00
Val loss: 1.03
Epoch 15/15
---------------
Train loss: 0.00
Val loss: 1.37


2023/11/29 00:22:16 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.7.1/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.


Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]

In [0]:
# Experiment 4_8
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 200, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 2, add_loss_criteria_weights = False, exp_name = 'MobileNetV2 E_4_8')

Epoch 1/15
---------------
Train loss: 0.13
Val loss: 1.24
Epoch 2/15
---------------
Train loss: 0.06
Val loss: 0.42
Epoch 3/15
---------------
Train loss: 0.03
Val loss: 0.78
Epoch 4/15
---------------
Train loss: 0.02
Val loss: 1.01
Epoch 5/15
---------------
Train loss: 0.02
Val loss: 1.48
Epoch 6/15
---------------
Train loss: 0.06
Val loss: 1.58
Epoch 7/15
---------------
Train loss: 0.04
Val loss: 0.66
Epoch 8/15
---------------
Train loss: 0.01
Val loss: 1.61
Epoch 9/15
---------------
Train loss: 0.03
Val loss: 0.64
Epoch 10/15
---------------
Train loss: 0.01
Val loss: 2.35
Epoch 11/15
---------------
Train loss: 0.02
Val loss: 2.85
Epoch 12/15
---------------
Train loss: 0.05
Val loss: 0.51
Epoch 13/15
---------------
Train loss: 0.04
Val loss: 0.55
Epoch 14/15
---------------
Train loss: 0.01
Val loss: 1.71
Epoch 15/15
---------------
Train loss: 0.02
Val loss: 0.36


2023/11/29 01:05:37 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.7.1/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.


Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]

In [0]:
# Experiment 4_9
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 50, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 1, add_loss_criteria_weights = False, exp_name = 'MobileNetV2 E_4_9')

Epoch 1/15
---------------
Train loss: 0.26
Val loss: 0.46
Epoch 2/15
---------------
Train loss: 0.16
Val loss: 0.54
Epoch 3/15
---------------
Train loss: 0.13
Val loss: 0.53
Epoch 4/15
---------------
Train loss: 0.14
Val loss: 0.41
Epoch 5/15
---------------
Train loss: 0.14
Val loss: 0.56
Epoch 6/15
---------------
Train loss: 0.14
Val loss: 0.36
Epoch 7/15
---------------
Train loss: 0.12
Val loss: 0.49
Epoch 8/15
---------------
Train loss: 0.12
Val loss: 0.43
Epoch 9/15
---------------
Train loss: 0.11
Val loss: 0.46
Epoch 10/15
---------------
Train loss: 0.10
Val loss: 0.41
Epoch 11/15
---------------
Train loss: 0.11
Val loss: 0.23
Epoch 12/15
---------------
Train loss: 0.11
Val loss: 0.58
Epoch 13/15
---------------
Train loss: 0.11
Val loss: 0.48
Epoch 14/15
---------------
Train loss: 0.11
Val loss: 0.35
Epoch 15/15
---------------
Train loss: 0.11
Val loss: 0.49


2023/11/29 01:47:03 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.7.1/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.


Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]

In [0]:
# Experiment 4_10
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 100, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 1, add_loss_criteria_weights = False, exp_name = 'MobileNetV2 E_4_10')

Epoch 1/15
---------------
Train loss: 0.33
Val loss: 0.96
Epoch 2/15
---------------
Train loss: 0.17
Val loss: 0.56
Epoch 3/15
---------------
Train loss: 0.15
Val loss: 0.51
Epoch 4/15
---------------
Train loss: 0.14
Val loss: 0.52
Epoch 5/15
---------------
Train loss: 0.14
Val loss: 0.37
Epoch 6/15
---------------
Train loss: 0.13
Val loss: 0.42
Epoch 7/15
---------------
Train loss: 0.12
Val loss: 0.51
Epoch 8/15
---------------
Train loss: 0.12
Val loss: 0.24
Epoch 9/15
---------------
Train loss: 0.12
Val loss: 0.39
Epoch 10/15
---------------
Train loss: 0.11
Val loss: 0.27
Epoch 11/15
---------------
Train loss: 0.11
Val loss: 0.40
Epoch 12/15
---------------
Train loss: 0.11
Val loss: 0.48
Epoch 13/15
---------------
Train loss: 0.11
Val loss: 0.56
Epoch 14/15
---------------
Train loss: 0.10
Val loss: 0.42
Epoch 15/15
---------------
Train loss: 0.10
Val loss: 0.31


2023/11/29 02:30:43 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.7.1/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.


Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]

In [0]:
# Experiment 4_11
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 150, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 1, add_loss_criteria_weights = False, exp_name = 'MobileNetV2 E_4_11')

Epoch 1/15
---------------
Train loss: 0.38
Val loss: 0.47
Epoch 2/15
---------------
Train loss: 0.19
Val loss: 0.25
Epoch 3/15
---------------
Train loss: 0.16
Val loss: 0.14
Epoch 4/15
---------------
Train loss: 0.14
Val loss: 0.13
Epoch 5/15
---------------
Train loss: 0.14
Val loss: 0.13
Epoch 6/15
---------------
Train loss: 0.13
Val loss: 0.12
Epoch 7/15
---------------
Train loss: 0.13
Val loss: 0.13
Epoch 8/15
---------------
Train loss: 0.13
Val loss: 0.11
Epoch 9/15
---------------
Train loss: 0.11
Val loss: 0.12
Epoch 10/15
---------------
Train loss: 0.12
Val loss: 0.11
Epoch 11/15
---------------
Train loss: 0.11
Val loss: 0.11
Epoch 12/15
---------------
Train loss: 0.11
Val loss: 0.11
Epoch 13/15
---------------
Train loss: 0.10
Val loss: 0.11
Epoch 14/15
---------------
Train loss: 0.10
Val loss: 0.12
Early stopping triggered after 3 epochs without improvement


2023/11/29 04:14:16 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/repl_tmp_data/ReplId-52341-9cf7e-13843-0/tmpdiwll5_h/model/data, flavor: pytorch), fall back to return ['torch==2.0.1', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback.


Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]

In [0]:
# Experiment 4_12
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 200, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 1, add_loss_criteria_weights = False, exp_name = 'MobileNetV2 E_4_12')

Epoch 1/15
---------------
Train loss: 0.41
Val loss: 0.51
Epoch 2/15
---------------
Train loss: 0.21
Val loss: 0.39
Epoch 3/15
---------------
Train loss: 0.17
Val loss: 0.19
Epoch 4/15
---------------
Train loss: 0.15
Val loss: 0.14
Epoch 5/15
---------------
Train loss: 0.14
Val loss: 0.13
Epoch 6/15
---------------
Train loss: 0.14
Val loss: 0.12
Epoch 7/15
---------------
Train loss: 0.13
Val loss: 0.12
Epoch 8/15
---------------
Train loss: 0.13
Val loss: 0.12
Epoch 9/15
---------------
Train loss: 0.12
Val loss: 0.11
Epoch 10/15
---------------
Train loss: 0.13
Val loss: 0.11
Epoch 11/15
---------------
Train loss: 0.12
Val loss: 0.11
Epoch 12/15
---------------
Train loss: 0.11
Val loss: 0.11
Epoch 13/15
---------------
Train loss: 0.11
Val loss: 0.10
Epoch 14/15
---------------
Train loss: 0.11
Val loss: 0.10
Epoch 15/15
---------------
Train loss: 0.11
Val loss: 0.10


2023/11/29 05:02:22 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/repl_tmp_data/ReplId-52341-9cf7e-13843-0/tmpod2gr6y5/model/data, flavor: pytorch), fall back to return ['torch==2.0.1', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback.


Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]

In [0]:
# Experiment 4_13
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 50, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 0, add_loss_criteria_weights = True, exp_name = 'MobileNetV2 E_4_13')

Epoch 1/15
---------------
Train loss: 0.14
Val loss: 0.10
Epoch 2/15
---------------
Train loss: 0.07
Val loss: 0.06
Epoch 3/15
---------------
Train loss: 0.06
Val loss: 0.08
Epoch 4/15
---------------
Train loss: 0.05
Val loss: 0.74
Epoch 5/15
---------------
Train loss: 0.04
Val loss: 0.16
Early stopping triggered after 3 epochs without improvement


2023/11/29 05:16:59 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/repl_tmp_data/ReplId-52341-9cf7e-13843-0/tmpamfy8vsd/model/data, flavor: pytorch), fall back to return ['torch==2.0.1', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback.


Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]

In [0]:
# Experiment 4_14
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 100, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 0, add_loss_criteria_weights = True, exp_name = 'MobileNetV2 E_4_14')

Epoch 1/15
---------------
Train loss: 0.15
Val loss: 0.19
Epoch 2/15
---------------
Train loss: 0.06
Val loss: 0.07
Epoch 3/15
---------------
Train loss: 0.03
Val loss: 0.09
Epoch 4/15
---------------
Train loss: 0.02
Val loss: 0.07
Epoch 5/15
---------------
Train loss: 0.01
Val loss: 0.06
Epoch 6/15
---------------
Train loss: 0.02
Val loss: 0.15
Epoch 7/15
---------------
Train loss: 0.03
Val loss: 0.08
Epoch 8/15
---------------
Train loss: 0.01
Val loss: 0.07
Early stopping triggered after 3 epochs without improvement


2023/11/29 05:41:35 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/repl_tmp_data/ReplId-52341-9cf7e-13843-0/tmpv6pn12sh/model/data, flavor: pytorch), fall back to return ['torch==2.0.1', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback.


Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]

In [0]:
# Experiment 4_15
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 150, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 0, add_loss_criteria_weights = True, exp_name = 'MobileNetV2 E_4_15')

Epoch 1/15
---------------
Train loss: 0.14
Val loss: 0.07
Epoch 2/15
---------------
Train loss: 0.05
Val loss: 0.08
Epoch 3/15
---------------
Train loss: 0.03
Val loss: 0.08
Epoch 4/15
---------------
Train loss: 0.01
Val loss: 0.08
Early stopping triggered after 3 epochs without improvement


2023/11/29 05:53:13 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/repl_tmp_data/ReplId-52341-9cf7e-13843-0/tmpbon9ku0c/model/data, flavor: pytorch), fall back to return ['torch==2.0.1', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback.


Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]

In [0]:
# Experiment 4_16
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 200, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 0, add_loss_criteria_weights = True, exp_name = 'MobileNetV2 E_4_16')

Epoch 1/15
---------------
Train loss: 0.19
Val loss: 0.23
Epoch 2/15
---------------
Train loss: 0.05
Val loss: 0.08
Epoch 3/15
---------------
Train loss: 0.02
Val loss: 0.06
Epoch 4/15
---------------
Train loss: 0.01
Val loss: 0.06
Epoch 5/15
---------------
Train loss: 0.01
Val loss: 0.16
Epoch 6/15
---------------
Train loss: 0.01
Val loss: 0.12
Early stopping triggered after 3 epochs without improvement


2023/11/29 06:10:24 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/repl_tmp_data/ReplId-52341-9cf7e-13843-0/tmpfey36dcc/model/data, flavor: pytorch), fall back to return ['torch==2.0.1', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback.


Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]

In [0]:
# Experiment 4_17
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 50, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 2, add_loss_criteria_weights = True, exp_name = 'MobileNetV2 E_4_17')

Epoch 1/15
---------------
Train loss: 0.14
Val loss: 0.09
Epoch 2/15
---------------
Train loss: 0.06
Val loss: 0.13
Epoch 3/15
---------------
Train loss: 0.07
Val loss: 0.11
Epoch 4/15
---------------
Train loss: 0.04
Val loss: 0.09
Epoch 5/15
---------------
Train loss: 0.02
Val loss: 0.16
Epoch 6/15
---------------
Train loss: 0.03
Val loss: 0.49
Epoch 7/15
---------------
Train loss: 0.03
Val loss: 0.09
Epoch 8/15
---------------
Train loss: 0.02
Val loss: 0.13
Epoch 9/15
---------------
Train loss: 0.03
Val loss: 0.10
Epoch 10/15
---------------
Train loss: 0.02
Val loss: 0.10
Early stopping triggered after 3 epochs without improvement


2023/11/29 06:38:56 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/repl_tmp_data/ReplId-52341-9cf7e-13843-0/tmp15au32l_/model/data, flavor: pytorch), fall back to return ['torch==2.0.1', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback.


Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]

In [0]:
# Experiment 4_18
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 100, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 2, add_loss_criteria_weights = True, exp_name = 'MobileNetV2 E_4_18')

Epoch 1/15
---------------
Train loss: 0.16
Val loss: 0.13
Epoch 2/15
---------------
Train loss: 0.04
Val loss: 0.08
Epoch 3/15
---------------
Train loss: 0.03
Val loss: 0.09
Epoch 4/15
---------------
Train loss: 0.03
Val loss: 0.07
Epoch 5/15
---------------
Train loss: 0.02
Val loss: 0.05
Epoch 6/15
---------------
Train loss: 0.01
Val loss: 0.07
Epoch 7/15
---------------
Train loss: 0.01
Val loss: 0.10
Epoch 8/15
---------------
Train loss: 0.02
Val loss: 0.11
Early stopping triggered after 3 epochs without improvement


2023/11/29 07:01:33 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/repl_tmp_data/ReplId-52341-9cf7e-13843-0/tmpel7aca9b/model/data, flavor: pytorch), fall back to return ['torch==2.0.1', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback.


Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]

In [0]:
# Experiment 4_19
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 150, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 2, add_loss_criteria_weights = True, exp_name = 'MobileNetV2 E_4_19')

Epoch 1/15
---------------
Train loss: 0.15
Val loss: 0.12
Epoch 2/15
---------------
Train loss: 0.06
Val loss: 0.11
Epoch 3/15
---------------
Train loss: 0.02
Val loss: 0.07
Epoch 4/15
---------------
Train loss: 0.01
Val loss: 0.09
Epoch 5/15
---------------
Train loss: 0.02
Val loss: 0.08
Epoch 6/15
---------------
Train loss: 0.01
Val loss: 0.07
Epoch 7/15
---------------
Train loss: 0.00
Val loss: 0.06
Epoch 8/15
---------------
Train loss: 0.00
Val loss: 0.06
Epoch 9/15
---------------
Train loss: 0.00
Val loss: 0.07
Epoch 10/15
---------------
Train loss: 0.00
Val loss: 0.12
Epoch 11/15
---------------
Train loss: 0.01
Val loss: 0.12
Early stopping triggered after 3 epochs without improvement


2023/11/29 07:32:48 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/repl_tmp_data/ReplId-52341-9cf7e-13843-0/tmpotm5btxg/model/data, flavor: pytorch), fall back to return ['torch==2.0.1', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback.


Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]

In [0]:
# Experiment 4_20
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 200, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 2, add_loss_criteria_weights = True, exp_name = 'MobileNetV2 E_4_20')

Epoch 1/15
---------------
Train loss: 0.15
Val loss: 0.07
Epoch 2/15
---------------
Train loss: 0.04
Val loss: 0.05
Epoch 3/15
---------------
Train loss: 0.03
Val loss: 0.10
Epoch 4/15
---------------
Train loss: 0.01
Val loss: 0.09
Epoch 5/15
---------------
Train loss: 0.02
Val loss: 0.11
Early stopping triggered after 3 epochs without improvement


2023/11/29 07:47:11 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/repl_tmp_data/ReplId-52341-9cf7e-13843-0/tmpha8gc7ka/model/data, flavor: pytorch), fall back to return ['torch==2.0.1', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback.


Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]

In [0]:
# Experiment 4_21
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 50, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 1, add_loss_criteria_weights = True, exp_name = 'MobileNetV2 E_4_21')

Epoch 1/15
---------------
Train loss: 0.30
Val loss: 0.17
Epoch 2/15
---------------
Train loss: 0.19
Val loss: 0.15
Epoch 3/15
---------------
Train loss: 0.17
Val loss: 0.14
Epoch 4/15
---------------
Train loss: 0.15
Val loss: 0.15
Epoch 5/15
---------------
Train loss: 0.14
Val loss: 0.14
Epoch 6/15
---------------
Train loss: 0.13
Val loss: 0.13
Epoch 7/15
---------------
Train loss: 0.13
Val loss: 0.13
Epoch 8/15
---------------
Train loss: 0.13
Val loss: 0.12
Epoch 9/15
---------------
Train loss: 0.14
Val loss: 0.11
Epoch 10/15
---------------
Train loss: 0.12
Val loss: 0.12
Epoch 11/15
---------------
Train loss: 0.13
Val loss: 0.11
Epoch 12/15
---------------
Train loss: 0.12
Val loss: 0.12
Epoch 13/15
---------------
Train loss: 0.11
Val loss: 0.11
Epoch 14/15
---------------
Train loss: 0.12
Val loss: 0.11
Epoch 15/15
---------------
Train loss: 0.11
Val loss: 0.11


2023/11/29 08:28:09 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/repl_tmp_data/ReplId-52341-9cf7e-13843-0/tmp_osy5d3m/model/data, flavor: pytorch), fall back to return ['torch==2.0.1', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback.


Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]

In [0]:
# Experiment 4_22
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 100, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 1, add_loss_criteria_weights = True, exp_name = 'MobileNetV2 E_4_22')

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 105MB/s] 


Epoch 1/15
---------------
Train loss: 0.37
Val loss: 0.40
Epoch 2/15
---------------
Train loss: 0.19
Val loss: 0.19
Epoch 3/15
---------------
Train loss: 0.17
Val loss: 0.18
Epoch 4/15
---------------
Train loss: 0.15
Val loss: 0.17
Epoch 5/15
---------------
Train loss: 0.14
Val loss: 0.17
Epoch 6/15
---------------
Train loss: 0.14
Val loss: 0.16
Epoch 7/15
---------------
Train loss: 0.14
Val loss: 0.16
Epoch 8/15
---------------
Train loss: 0.13
Val loss: 0.18
Epoch 9/15
---------------
Train loss: 0.12
Val loss: 0.16
Epoch 10/15
---------------
Train loss: 0.13
Val loss: 0.16
Early stopping triggered after 3 epochs without improvement


/databricks/python/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]

In [0]:
# Experiment 4_23
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 150, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 1, add_loss_criteria_weights = True, exp_name = 'MobileNetV2 E_4_23')

Epoch 1/15
---------------
Train loss: 0.39
Val loss: 0.68
Epoch 2/15
---------------
Train loss: 0.19
Val loss: 0.26
Epoch 3/15
---------------
Train loss: 0.17
Val loss: 0.17
Epoch 4/15
---------------
Train loss: 0.16
Val loss: 0.16
Epoch 5/15
---------------
Train loss: 0.15
Val loss: 0.15
Epoch 6/15
---------------
Train loss: 0.15
Val loss: 0.15
Epoch 7/15
---------------
Train loss: 0.13
Val loss: 0.14
Epoch 8/15
---------------
Train loss: 0.13
Val loss: 0.14
Epoch 9/15
---------------
Train loss: 0.14
Val loss: 0.14
Epoch 10/15
---------------
Train loss: 0.12
Val loss: 0.14
Epoch 11/15
---------------
Train loss: 0.13
Val loss: 0.16
Epoch 12/15
---------------
Train loss: 0.13
Val loss: 0.13
Epoch 13/15
---------------
Train loss: 0.11
Val loss: 0.13
Epoch 14/15
---------------
Train loss: 0.12
Val loss: 0.14
Epoch 15/15
---------------
Train loss: 0.11
Val loss: 0.13


Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]

In [0]:
# Experiment 4_24
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 200, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 1, add_loss_criteria_weights = True, exp_name = 'MobileNetV2 E_4_24')

Epoch 1/15
---------------
Train loss: 0.48
Val loss: 0.59
Epoch 2/15
---------------
Train loss: 0.24
Val loss: 0.54
Epoch 3/15
---------------
Train loss: 0.20
Val loss: 0.22
Epoch 4/15
---------------
Train loss: 0.17
Val loss: 0.14
Epoch 5/15
---------------
Train loss: 0.17
Val loss: 0.14
Epoch 6/15
---------------
Train loss: 0.16
Val loss: 0.14
Epoch 7/15
---------------
Train loss: 0.16
Val loss: 0.13
Epoch 8/15
---------------
Train loss: 0.15
Val loss: 0.12
Epoch 9/15
---------------
Train loss: 0.15
Val loss: 0.12
Epoch 10/15
---------------
Train loss: 0.14
Val loss: 0.12
Epoch 11/15
---------------
Train loss: 0.13
Val loss: 0.12
Epoch 12/15
---------------
Train loss: 0.13
Val loss: 0.11
Epoch 13/15
---------------
Train loss: 0.13
Val loss: 0.11
Epoch 14/15
---------------
Train loss: 0.13
Val loss: 0.12
Epoch 15/15
---------------
Train loss: 0.13
Val loss: 0.11


Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]